In [16]:
import numpy as np
import pandas as pd
import category_encoders
import sklearn
from sklearn.preprocessing import OneHotEncoder
from category_encoders import LeaveOneOutEncoder

In [17]:
test_set_values = pd.read_csv("/Users/azeemmuzammil/Documents/Aca/pump-it-up/data_set/test_set_values.csv")
training_set_labels = pd.read_csv("/Users/azeemmuzammil/Documents/Aca/pump-it-up/data_set/training_set_labels.csv")
training_set_values = pd.read_csv("/Users/azeemmuzammil/Documents/Aca/pump-it-up/data_set/training_set_values.csv")

In [18]:
training_set = pd.merge(training_set_values , training_set_labels , on ="id")

In [21]:
for column in training_set.columns[training_set.isna().sum() > 0]:
        mode = training_set[column].mode()[0]
        training_set[column].fillna(value = mode, inplace = True)
for column in test_set_values.columns[test_set_values.isna().sum() > 0]:
        mode = test_set_values[column].mode()[0]
        test_set_values[column].fillna(value = mode, inplace = True)

In [22]:
training_set.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [23]:
train_set_drop_columns = [ 'region_code', 'recorded_by','funder', 'num_private', 'subvillage', 'source_type','scheme_name', 'payment_type', 'waterpoint_type', 'quantity_group']
test_set_drop_columns = [ 'num_private', 'recorded_by', 'waterpoint_type', 'subvillage', 'region_code','funder', 'source_type','scheme_name', 'payment_type', 'quantity_group']
training_set.drop(columns = train_set_drop_columns ,inplace = True)
test_set_values.drop(columns = test_set_drop_columns, inplace = True)

In [24]:
training_set['status_group'] = training_set['status_group'].map({'functional': 2, 'functional needs repair': 1, 'non functional': 0})

In [25]:
training_set['date_recorded'] = pd.to_datetime(training_set['date_recorded'])
training_set['year_recorded'] = training_set['date_recorded'].dt.year
training_set['month_recorded'] = training_set['date_recorded'].dt.month
training_set.drop(columns = 'date_recorded', inplace = True)

test_set_values['date_recorded'] = pd.to_datetime(test_set_values['date_recorded'])
test_set_values['year_recorded'] = test_set_values['date_recorded'].dt.year
test_set_values['month_recorded'] = test_set_values['date_recorded'].dt.month
test_set_values.drop(columns = 'date_recorded', inplace = True)

In [26]:
target_value = 'status_group'
encoder = LeaveOneOutEncoder()
encode_columns = ['wpt_name', 'basin', 'region', 'district_code', 'lga', 'ward', 'scheme_management','installer','source',
                    'extraction_type', 'extraction_type_group', 'extraction_type_class','management', 'payment', 'water_quality', 'management_group', 'quality_group', 
        'quantity','source_class', 'waterpoint_type_group'] 

for column in encode_columns:
    training_set[str(column) + '_encoded'] = encoder.fit_transform(training_set[column].values, training_set[target_value])
    test_set_values[str(column) + '_encoded'] = encoder.transform(test_set_values[column].values)
    training_set.drop(columns=column, inplace=True) 
    test_set_values.drop(columns=column, inplace=True)

In [27]:
training_set.head()

,id,amount_tsh,gps_height,longitude,latitude,population,public_meeting,permit,construction_year,status_group,...,extraction_type_group_encoded,extraction_type_class_encoded,management_encoded,payment_encoded,water_quality_encoded,management_group_encoded,quality_group_encoded,quantity_encoded,source_class_encoded,waterpoint_type_group_encoded
0,69572,6000.0,1390,34.938093,-9.856322,109,True,False,1999,2,...,1.299339,1.299339,1.077347,1.572370,1.208690,1.150870,1.208690,1.376947,1.144476,1.236945
1,8776,0.0,1399,34.698766,-2.147466,280,True,True,2010,2,...,1.299339,1.299339,1.297974,0.973015,1.208690,1.150870,1.208690,1.142253,1.208149,1.236945
2,34310,25.0,686,37.460664,-3.821329,250,True,True,2009,2,...,1.299339,1.299339,1.077347,1.401046,1.208690,1.150870,1.208690,1.376947,1.208149,1.236945
3,67743,0.0,263,38.486161,-11.155298,58,True,True,1986,0,...,1.118647,1.118647,1.077396,0.973093,1.208729,1.150908,1.208729,0.056205,1.144520,1.237003
4,19728,0.0,0,31.130847,-1.825359,0,True,True,0,2,...,1.299339,1.299339,1.260973,0.973015,1.208690,1.178344,1.208690,1.250679,1.208149,1.236945
